<h1><center>Estonia Disaster</center></h1>

<center><img src="https://i.pinimg.com/originals/fc/35/72/fc357254541c2886a4a8722cbfa9631e.png" width=400></center>
<p>&#160;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<p>
    <p>&#160;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<p>
       
                

        
Hello, today we will take a look at the problems requested from the data set we have.Hello, today we will take a look at the problems requested from the data set we have.
- Who's more likely to survive the sinking based on data?

- Is age an indicator for survival?

- Is gender an indicator for survival?

- Did the crew aboard have a higher chance of survival than passengers?
        
        
We will address these problems and solve them.

# Loading And Cleaning Data

In [ ]:
import numpy as np 
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
passanger = pd.read_csv('../input/passenger-list-for-the-estonia-ferry-disaster/estonia-passenger-list.csv')
data = passanger.copy()
data.head()

In [ ]:
# We check missing values
data.isnull().any()

Thats great,so clean data

# Problems

In this section, we will visualize some problems that stuck in my mind.

In [ ]:
# How many men and women were on board
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(10,8))
sns.countplot(data['Sex'],palette='coolwarm');
print(data['Sex'].value_counts())

Gender distribution of passenger numbers is very close to each other

**So let's ask one thing, according to these gender distributions, are women more saved or men?**

In [ ]:
plt.figure(figsize=(10,8))
sns.countplot(data['Sex'],hue=data['Survived'],palette='BuGn_r')
plt.xlabel('Sex')
plt.ylabel('Number Of People')
plt.title('Survived Status by Gender');

**According to the table here, there have been more survivors than men, I don't know exactly what could be the reason.**

**Now tell me my question, how many people survived as team or passenger.**

In [ ]:
plt.figure(figsize=(10,8))
sns.countplot(data['Survived'],hue=data['Category'],palette='vlag')
plt.xlabel('Survived')
plt.ylabel('Number of people')
plt.title('Crew Or Passenger Survived Status');

I can comment on this situation like this;


The number of passengers who died is high, but this is because the participation of passengers is  as high as the team.


But even in this situation, can we say that too many people from the team died, they fought with heart and soul to save the passengers?

**I also want to handle the following situation, Let's see where the passengers who joined the ship.**

In [ ]:
plt.figure(figsize=(15,10))
sns.countplot(data['Country'],hue=data['Category'],palette='rocket')
plt.xlabel('Country')
plt.ylabel('Number Of People')
plt.title('Where Do The Team Or Passengers Join From')
plt.legend(loc='upper right');

**Most of the participating passengers were from Sweden, but apparently the team joined from Estonia.**

**Total survivors or those who could not.**

In [ ]:
plt.figure(figsize=(10,8))
labels = ['No Survive','Survive']
explode = (0, 0.1) 
plt.pie(data['Survived'].value_counts(),labels=labels,explode=explode,autopct='%1.1f%%',shadow=True, startangle=180,colors=['#F3CFB3','#F3C4BF'])
plt.title('Survival Rate')
plt.legend();

That's all for the heuristic analysis, now let's set up a classification model with decision trees.

# Decision Tree
Decision tree is a decision support tool that uses a tree-like decision model and possible outcomes, including chance-like outcomes, resource costs, and benefits.

I think you can help us with this problem.

<center><img src="https://i.ytimg.com/vi/7VeUPuFGJHk/maxresdefault.jpg" width=600></center>

In [ ]:
# Datasets Seperation
data['Category'] = [1 if i.strip() == 'P'else 0 for i in data.Category]
data['Sex'] = [1 if i.strip() == 'M'else 0 for i in data.Sex]
y = data['Survived']
x = data.loc[:,['Sex','Age','Category']]
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.30,random_state=42)
from sklearn.tree import DecisionTreeClassifier
DC = DecisionTreeClassifier()
DC.fit(x_train,y_train)

In [ ]:
from sklearn.metrics import accuracy_score
y_pred = DC.predict(x_test)
print('Basit Accuracy Score:',accuracy_score(y_test,y_pred))

# Model Tuning

In [ ]:
from sklearn.model_selection import GridSearchCV
dc_params = {'criterion':['gini','entropy'],
            'splitter':['best','random'],
            'max_depth':np.arange(1,10),
            'min_samples_split':np.arange(1,10),
            'min_samples_leaf':np.arange(1,10),
            'max_features':['auto','sqrt','log2']}
dc = DecisionTreeClassifier()
dc_cv = GridSearchCV(dc,dc_params,cv=10,n_jobs=-1,verbose=2)
dc_cv.fit(x_train,y_train)

# Tuned Model Create

In [ ]:
dc_tuned = DecisionTreeClassifier(criterion=dc_cv.best_params_['criterion'],
                                 max_depth=dc_cv.best_params_['max_depth'],
                                 max_features=dc_cv.best_params_['max_features'],
                                 min_samples_leaf=dc_cv.best_params_['min_samples_leaf'],
                                 min_samples_split=dc_cv.best_params_['min_samples_split'],
                                 splitter=dc_cv.best_params_['splitter']).fit(x_train,y_train)

In [ ]:
y_pred = dc_tuned.predict(x_test)
print('Verified Accuracy Score:',accuracy_score(y_test,y_pred))

# Importance Of Variables

In [ ]:
Importance = pd.DataFrame({'Importance':dc_tuned.feature_importances_*100},
                         index=x_train.columns)
x = Importance.sort_values(by='Importance',axis=0,ascending=True).plot(kind='barh',color='#9B5F2B')

Here we will understand that the variables that are important to us are the variables of age and gender.

**Thank you for reading, I wish you a pleasant day**